In [ ]:
import func_text_gen as f

In [ ]:
Yeats_poems, Yeats_title = f.split_data('Poems/Yeats.txt')
Emily_poems, Emily_title = f.split_data('Poems/Emily_Dickinson.txt')
Gibran_poems, Gibran_title = f.split_data('Poems/Gibran.txt')

In [ ]:
yeats = pd.DataFrame({'poems':Yeats_poems,'title':Yeats_title})
emily = pd.DataFrame({'poems':Emily_poems,'title':Emily_title})
gibran = pd.DataFrame({'poems':Gibran_poems,'title':Gibran_title})

In [ ]:
df = f.pd.read_csv("all.csv")
df['content'] = df['content'].str.lower()
df.rename(columns={'content':'poems','poem name': 'title'}, inplace=True)

In [ ]:
all_df = [df,yeats, gibran, emily]
new_df = f.pd.concat(all_df)
new_df["poems"] = new_df['poems'].str.lower()

In [ ]:
### Insert documents in a MongoDB database

from pymongo import MongoClient

client = MongoClient()
db = client.all_poems
collection = db.new_files
collection.insert_many(new_df.to_dict('records'))

### Shows 10 topics present in the text

In [ ]:
gen= collection.find()
f.topic_modeling(gen, n_components=10)

### Labels of pattern are formed. Model Architecture is also defined

In [ ]:
gen = collection.find()
X_modified, Y_modified, X, characters = f.pattern_and_label(gen, seq_length=100)
model_arch = f.model_generation(X_modified, Y_modified, size=700)

### Fits the model to the training and labels

In [ ]:
fit_model = f.model_fit(model_arch, X_modified, Y_modified, epochs=5)

### Generates 500 new characters after a random seed

In [ ]:
f.text_generator(X, fit_model, characters=characters, num_characters=500)

### Check the sentiment of the poem generated

In [ ]:
s = text_generator(X, fit_model, characters=characters, num_characters=500)
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
pol = sid.polarity_scores(s)
pol